In [4]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

dataDir = "/code/archive/garbage_classification/Garbage_classification"

imgHeight = 224
imgWidth = 224
batchSize = 32

trainDs = tf.keras.utils.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  labels='inferred',
  subset="training",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize)

valDs = tf.keras.utils.image_dataset_from_directory(
  dataDir,
  validation_split=0.2,
  labels='inferred',
  subset="validation",
  seed=123,
  image_size=(imgHeight, imgWidth),
  batch_size=batchSize)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(.5, .2),
  # tf.keras.layers.RandomContrast(0.2),
])

mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

feature_extractor_model = mobilenet_v2

IMAGE_SHAPE = (imgHeight, imgWidth)

# Apply normalization to images
normalization_layer = tf.keras.layers.Rescaling(1./255)
trainDs = trainDs.map(lambda x, y: (normalization_layer(x), y)) 
valDs = valDs.map(lambda x, y: (normalization_layer(x), y)) 

AUTOTUNE = tf.data.AUTOTUNE
trainDs = trainDs.cache().prefetch(buffer_size=AUTOTUNE)
valDs = valDs.cache().prefetch(buffer_size=AUTOTUNE)

resize_and_rescale = tf.keras.Sequential([
  tf.keras.layers.Resizing(imgHeight,imgWidth),
])

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(imgHeight, imgWidth, 3),
    trainable=False)

feature_extractor_layer.trainable = False


model = tf.keras.Sequential([
  resize_and_rescale,
  data_augmentation,
  feature_extractor_layer,
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])
# model.summary()




Found 2390 files belonging to 5 classes.
Using 1912 files for training.
Found 2390 files belonging to 5 classes.
Using 478 files for validation.


In [8]:
# Fit model

history = model.fit(trainDs,
epochs=15,
validation_data=valDs)

# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

Epoch 1/15
60/60 [==============================] - 27s 455ms/step - loss: 0.2094 - acc: 0.9189 - val_loss: 0.3697 - val_acc: 0.8598
Epoch 2/15
60/60 [==============================] - 26s 438ms/step - loss: 0.2203 - acc: 0.9210 - val_loss: 0.3925 - val_acc: 0.8577
Epoch 3/15
60/60 [==============================] - 25s 422ms/step - loss: 0.1973 - acc: 0.9294 - val_loss: 0.3895 - val_acc: 0.8766
Epoch 4/15
60/60 [==============================] - 25s 422ms/step - loss: 0.2014 - acc: 0.9221 - val_loss: 0.3658 - val_acc: 0.8745
Epoch 5/15
60/60 [==============================] - 25s 422ms/step - loss: 0.1714 - acc: 0.9399 - val_loss: 0.3718 - val_acc: 0.8682
Epoch 6/15
60/60 [==============================] - 25s 422ms/step - loss: 0.1852 - acc: 0.9346 - val_loss: 0.3779 - val_acc: 0.8703
Epoch 7/15
60/60 [==============================] - 26s 431ms/step - loss: 0.1768 - acc: 0.9372 - val_loss: 0.3816 - val_acc: 0.8619
Epoch 8/15
60/60 [==============================] - 26s 428ms/step - 

In [9]:
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])


KeyError: 'accuracy'

In [47]:
from PIL import Image
model.save('./saved_mode/garbage_class.h5')
# img = Image.open("/code/test_images/plastic_bottle_1.jpg").convert('L').resize((imgWidth,imgHeight), Image.ANTIALIAS)
# img = np.array(img)
# print(img.shape())

img = tf.io.read_file("/code/test_images/metal_4.jpg")
tensor = tf.io.decode_image(img, channels=3, dtype=tf.dtypes.float32)
tensor = tf.image.resize(tensor, [imgHeight, imgWidth])
input_tensor = tf.expand_dims(tensor, axis=0)

prediction = model.predict(input_tensor)
print(prediction)
maxValue = max(prediction[0])
print("Max probability",maxValue)

# Set classes labels
classes = ['Cardboard','Glass','Metal','Paper','Plastic']

print("Predicted index",np.where(prediction[0] == maxValue)[0][0])
print("Predicted class:",classes[np.where(prediction[0] == maxValue)[0][0]])
print(sum(prediction[0]))

1/1 [==============================] - 0s 53ms/step
[[1.5315803e-04 4.3285124e-02 9.5343727e-01 8.5262604e-05 3.0392420e-03]]
Max probability 0.95343727
Predicted index 2
Predicted class: Metal
1.0000000553773134
